# 🤖 Run CrewAI with Free Hugging Face Models (No OpenAI, No LiteLLM)

This notebook helps you understand **why CrewAI doesn’t work out-of-the-box** with free Hugging Face models — and how to still use it effectively.

We’ll explore:

❌ What fails  
✅ What works  
🛠️ Real fixes & workarounds  
📦 Colab-compatible implementation


In [ ]:
# @title
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown

display(HTML("<h2 style='color:#D32F2F'>❌ Challenges using Hugging Face Pipelines with CrewAI</h2>"))

# Point 1
point1 = widgets.Accordion(children=[widgets.Output()])
point1.set_title(0, "🚫 pipeline() doesn’t support .invoke() or chat-style outputs")
with point1.children[0]:
    display(Markdown("""
- `pipeline("text2text-generation")` is meant for simple prompts → outputs only.
- It doesn’t allow `.invoke()` or handle message formats like chat models do.
- CrewAI and LangChain expect `.invoke()` to work — so this setup fails silently.
    """))

# Point 2
point2 = widgets.Accordion(children=[widgets.Output()])
point2.set_title(0, "📦 Wrapping with LangChain’s BaseChatModel or CrewAI’s CustomLLM crashes")
with point2.children[0]:
    display(Markdown("""
- These wrappers assume the underlying model has a `.invoke()` method.
- `pipeline()` lacks that → wrapper methods call undefined functions.
- The result is **silent failure** — no output or hard-to-debug errors.
    """))

# Point 3
point3 = widgets.Accordion(children=[widgets.Output()])
point3.set_title(0, "⚠️LiteLLM errors")
with point3.children[0]:
    display(Markdown("""
Common runtime errors when using pipeline with LiteLLM:

- These errors indicate missing config or unsupported interfaces.
    """))

# Point 4
point4 = widgets.Accordion(children=[widgets.Output()])
point4.set_title(0, "🤖 Chat-capable models fail via .pipeline()")
with point4.children[0]:
    display(Markdown("""
- Even models like **Mistral**, **LLaMA 2 Chat**, or **Falcon** support chat — but **not via `pipeline()`**.
- Using `pipeline("text-generation")` removes message structure and context.
- Chat agents like CrewAI depend on `ChatModel`-style invocation and memory.
    """))

# Display all
display(point1, point2, point3, point4)



Accordion(children=(Output(),), _titles={'0': '🚫 pipeline() doesn’t support .invoke() or chat-style outputs'})

Accordion(children=(Output(),), _titles={'0': '📦 Wrapping with LangChain’s BaseChatModel or CrewAI’s CustomLLM…

Accordion(children=(Output(),), _titles={'0': '⚠️LiteLLM errors'})

Accordion(children=(Output(),), _titles={'0': '🤖 Chat-capable models fail via .pipeline()'})

In [ ]:
# @title
import ipywidgets as widgets
from IPython.display import display, Markdown

# Display the main title
display(HTML("<h2 style='color:#4CAF50'>🤖 CrewAI Hugging Face Compatibility Q&A</h2>"))

# Create a dropdown with questions
question = widgets.Dropdown(
    options=[
        "❓ Why does CrewAI need .invoke()?",
        "📦 What is AIMessage?",
        "⚠️ Why doesn’t HF pipeline work with CrewAI?",
        "🚫 Why does LiteLLM throw error?"
    ],
    description='📌 Question:',
    layout=widgets.Layout(width='70%')
)

output = widgets.Output()

def answer(change=None):
    output.clear_output()
    with output:
        q = question.value
        if "invoke" in q:
            display(Markdown(
                "### 🔄 Why `.invoke()`?\n"
                "CrewAI is built to simulate **chat-like conversations** with LLMs.\n\n"
                "- It sends prompts using `.invoke()` → expects structured replies.\n"
                "- The reply must be a special format: `AIMessage(content='...')`\n\n"
                "```python\ncrew_agent.invoke(\"Summarize this article\")\n# returns AIMessage(content='Here is the summary')\n```\n"
                "💡 **Note:** Simple models like `pipeline()` don’t support this flow."
            ))
        elif "AIMessage" in q:
            display(Markdown(
                "### 💬 What is `AIMessage`?\n"
                "`AIMessage` is a structured message that CrewAI understands — like a chat bubble.\n\n"
                "- It's used to **standardize responses** from all LLMs.\n"
                "- Helps CrewAI process replies like a conversation.\n\n"
                "```python\nfrom langchain.schema import AIMessage\nAIMessage(content='Here is your answer')\n```"
            ))
        elif "pipeline" in q:
            display(Markdown(
                "### ⚠️ Why Hugging Face `pipeline()` fails\n"
                "- `pipeline()` is designed for **simple text in → text out** tasks.\n"
                "- It doesn’t support chat structure or `.invoke()`\n"
                "- Wrapping it with LangChain’s `BaseChatModel` or CrewAI's LLM class? ❌ Crashes or returns raw strings.\n\n"
                "**🔍 Analogy:** It's like talking to a calculator — you say something, it replies — but it can’t remember context or chat with you."
            ))
        elif "LiteLLM" in q:
            display(Markdown(
                "### 🚫 Why LiteLLM Errors?\n"
                "**LiteLLM** connects CrewAI to 30+ model providers — but **free Hugging Face models** don’t play nice.\n\n"
                "- ❗ Error: `LLM Provider NOT provided`\n"
                "- ❗ Error: `BadRequestError`, `.pipe` not found\n\n"
                "**Why?**\n"
                "- `pipeline()` doesn’t expose `.invoke()` or `.call()`\n"
                "- LiteLLM expects a chat-style API interface\n\n"
                "**🛠️ Fix:**\n"
                "- Use a real chat model with `transformers` and `.generate()`\n"
                "- Or skip LiteLLM and run logic manually without `crew.kickoff()`"
            ))

# Display the first answer by default
answer()

# Attach the function to dropdown changes
question.observe(answer, names='value')

# Show the dropdown and answer box
display(question, output)


Dropdown(description='📌 Question:', layout=Layout(width='70%'), options=('❓ Why does CrewAI need .invoke()?', …

Output()

In [ ]:
!pip install crewai crewai-tools transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of chromadb to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looki

In [ ]:
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
from crewai import Agent, Task, Crew
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.language_models.chat_models import BaseChatModel
from typing import List, Optional
from pydantic import Field

# ✅ Fixed Custom LLM wrapper for CrewAI
class HFWrapper(BaseChatModel):
    pipe: any = Field()  # Declare this field so Pydantic doesn't complain

    def _call(self, prompt, stop=None):
        return self.pipe(prompt)[0]['generated_text']

    def invoke(self, input, config=None, **kwargs):
        content = input[0].content if isinstance(input, list) else input
        return AIMessage(content=self._call(content))

    def _generate(self, messages: List[HumanMessage], stop: Optional[List[str]] = None, **kwargs):
        return self._call(messages[0].content)

    @property
    def _llm_type(self):
        return "hf-wrapper"

# ✅ Load HF model
pipe = pipeline("text2text-generation", model="google/flan-t5-small")
llm = HFWrapper(pipe=pipe)  # Now we pass using field name

# ✅ Define CrewAI setup
agent = Agent(
    role="Summarizer",
    goal="Summarize text",
    backstory="Helps with text summarization",
    llm=llm
)

task = Task(
    description="Summarize: 'CrewAI does not natively support local Hugging Face models.'",
    expected_output="Short summary",
    agent=agent
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True
)
# ❗ CrewAI expects the LLM to support `.invoke()` and return `AIMessage`
# ❌ Hugging Face pipeline + wrapper lacks proper chat model behavior
# This causes LiteLLM inside CrewAI to raise: "LLM Provider NOT provided"
result = crew.kickoff()
print("\n✅ Result:\n", result)


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "pipe" in "HFWrapper" shadows an attribute in parent "BaseChatModel"
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:628: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 9eec05ee-16d9-4b5d-9cf4-6e352c885bab                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Provider List: https://docs.litellm.ai/docs/providers

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Summarizer                                                                                              │
│                                                                                                                 │
│  Task: Summarize: 'CrewAI does not natively support local Hugging Face models.'                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Provider List: https://docs.litellm.ai/docs/providers

...

ERROR:root:LiteLLM call failed: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=pipe=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7db6528dae50>
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers



 An unknown error occurred. Please check the details below.



╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 9bd7d34e-e8ca-4b3a-b3ed-7bed16c3d42d                                                                     │
│  Agent: Summarizer                                                                                              │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 9eec05ee-16d9-4b5d-9cf4-6e352c885bab                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=pipe=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7db6528dae50>
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers

# ✅ What Works

- Use **manual chaining** (not `crew.kickoff()`)
- Define `Agents`, `Tasks` using CrewAI
- Run actual logic using **plain functions or external orchestration**



In [ ]:
from transformers import pipeline

# Load local summarization model
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Simulated agents
def research_agent(article_text):
    key_points = [line.strip() for line in article_text.strip().split(".") if line.strip()]
    return key_points[:5]  # top 5 key points

def summarizer_agent(key_points):
    input_text = " ".join(key_points)
    summary = summarizer(input_text, max_length=100, min_length=30, do_sample=False)
    return summary[0]['summary_text']

def writer_agent(summary_text):
    return f"# Summary Report\n\n{summary_text}\n\n-- End of Report"

# Test input
article_text = """
CrewAI is an agent framework. It uses LiteLLM under the hood. Hugging Face pipelines don’t work natively.
You can create agents and tasks. But they expect chat-style models. Local HF models don't support invoke().
"""

# Manual chaining (bypassing Crew.kickoff)
points = research_agent(article_text)
summary = summarizer_agent(points)
final = writer_agent(summary)

print("✅ Final Output:\n", final)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Device set to use cpu
Your max_length is set to 100, but your input_length is only 40. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=20)


✅ Final Output:
 # Summary Report

 CrewAI is an agent framework that uses LiteLLM under the hood . Hugging Face pipelines don’t work natively You can create agents and tasks but they expect chat-style models .

-- End of Report
